In [145]:
%matplotlib inline
import pandas as pd
import numpy as np

In [146]:
data = pd.read_excel('Template1.xlsx',sheet_name = None)

In [147]:
xls = pd.ExcelFile('Template1.xlsx')
xls.sheet_names 
flow = data.get('Flow vs. Group')
flow.head()

,SN,Flow,Step Name,WS,Area,Step SEQ NO,STEP STATUS,REC TO SCHED?,EXPORT TO CRINV?
0,1,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,11-3DNS_SU31_DHF100,F11 WET PROCESS,100,ACTIVE,Y,Y
1,1,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,11-3DNS_SU3_DHF100,F11 WET PROCESS,100,ACTIVE,Y,N
2,1,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,11-3DNS_SU31_DHF_4,F11 WET PROCESS,100,ACTIVE,Y,N
3,2,100S PC,5200-53 GATE NITRIDE DEP 2 CMP,11-3EBRA_FRXX_BUFF,F11 CMP,200,ACTIVE,Y,N
4,2,100S PC,5200-53 GATE NITRIDE DEP 2 CMP,11-3EBRA_FRXS_BUFF,F11 CMP,200,ACTIVE,Y,Y


In [148]:
flow = flow[flow['Flow'] == '120S PC']

In [149]:
Qty = data.get('WS vs. Tool ID') 
WS_NAME = Qty['WS']
WS_NAME1 = []
for i in WS_NAME:
    #i.replace('\\xa0', '')
    WS_NAME1.append(i.replace('\xa0', ''))
print(len(WS_NAME1))

3780


In [150]:
G_WS = data.get('Group vs. WS')
G_WS1 = G_WS.set_index(['Capacity Group'],inplace = False)
WS_Group = []
for i in WS_NAME1:
    if i in G_WS1['WS'].tolist():
        group = G_WS1[G_WS1['WS'] ==  i].index.tolist()
    else:
        group = ['0']
    #group = G_WS[G_WS['WS'] ==  i].iat[0,0]
    WS_Group.extend(group)
    #print(group)
Qty['WS Group'] = WS_Group
Qty.head()

,MiCAP ID,WS,Tool ID,Location,WS Group
0,1195118,11-3HIKE_Q2LV_UDPLY,KISDB7VA00,7,11-DIF_UDPLY
1,1195119,11-3HIKE_Q2LV_BSIGE,KISDB7V300,7,11-DIF_BSIGEPLY
2,1195249,11-3TEL_SDRM_EBK,TTSDBC0130,C,11-DRY_SDRM_EBK
3,1195249,11-3TEL_SDRM_EBK,TTSDBC0120,C,11-DRY_SDRM_EBK
4,1195249,11-3TEL_SDRM_EBK,TTSDBC0110,C,11-DRY_SDRM_EBK


In [154]:
Qty.loc[3562,'Location']

'7'

In [155]:
step = []
for i in flow['WS']:
    if i in G_WS1['WS'].tolist():
        s = G_WS1[G_WS1['WS'] ==  i].index.tolist()
    else:
        s = ['0']
    step.extend(s)
print(step[:10])

['11-WET_SCRUB_DS', '11-WET_SCRUB_DS', '11-WET_SCRUB_DS', '11-WET_SCRUB_DS', '11-WET_SCRUB_DS', '11-WET_SCRUB_DS', '11-WET_SCRUB_DS', '11-WET_SCRUB_DS', '11-WET_SCRUB_DS', '11-WET_SCRUB_DS']


In [156]:
flow = flow.drop(['Area','Step SEQ NO','STEP STATUS','REC TO SCHED?','EXPORT TO CRINV?'], axis = 1)
#flow['WS Group'] = step
flow.head()

,SN,Flow,Step Name,WS
3827,1,120S PC,5420-88 PASS NITRIDE DRY ETCH SCRUB,11-3TEL_NS3S_SCRB_DS
3828,1,120S PC,5420-88 PASS NITRIDE DRY ETCH SCRUB,11-3DNS_SS33_SCRB_DS
3829,1,120S PC,5420-88 PASS NITRIDE DRY ETCH SCRUB,11-3DNS_SS32_SCRB_DS
3830,2,120S PC,5420-5C SPACER OXIDE DRY STRIP 2 SCRUB,11-3TEL_NS3S_SCRB_DS
3831,2,120S PC,5420-5C SPACER OXIDE DRY STRIP 2 SCRUB,11-3DNS_SS32_SCRB_DS


In [161]:
loc1 = G_WS1[G_WS1['WS'] == '11-3PSK_SVP_F_STR_IM' ].index.tolist()
loc2 = G_WS1[G_WS1['WS'] == '11-3PSK_SVP_F_STR_NI' ].index.tolist()
loc1.extend(loc2)
#print(loc1)
#WS = flow['WS Group'].tolist()
loc_change = []
for j in range(len(loc1)):
    loc_change.extend([i for i,x in enumerate(step) if x == loc1[j]])
loc_change_group = []
for k in loc_change:
    _WS_ = step[k-1:k+2] #提取索引
    loc_change_group.append(_WS_)
print(len(loc_change_group))
#print(loc1)
'''loc_origin = pd.DataFrame()
for i in range(len(loc1)):
    loc_origin = loc_origin.append(flow[flow['WS Group'] == loc1[i]])
loc_origin.head(100)'''

61


"loc_origin = pd.DataFrame()\nfor i in range(len(loc1)):\n    loc_origin = loc_origin.append(flow[flow['WS Group'] == loc1[i]])\nloc_origin.head(100)"

In [162]:
micap = [1351160,1351165,1351166,1351167,1351170,1351174]
c_loc = ['E','E','E','E','E','E','8','8','8','8','8','8','8','8','8','8','8','8']
change = pd.DataFrame()
for i in micap :
    change = pd.concat([change,Qty[Qty['MiCAP ID'] == i]],axis = 0)
print(change.index.tolist())
print(len(c_loc))

[3562, 3563, 3564, 3574, 3575, 3576, 3577, 3578, 3579, 3580, 3581, 3582, 3589, 3590, 3591, 3598, 3599, 3600]
18


In [163]:
for i in range(len(c_loc)):
    Qty.loc[change.index.tolist()[i],'Location'] = c_loc[i]
#print(Qty.loc[3577,'Location'])
#print(Qty.iloc[3562:3601,:])

In [164]:
Qty.loc[3562,'Location']

'E'

In [165]:
wafer = input('Please enter the quantity of wafers:')
wafer_m = []
for i in step:
    if i in WS_Group:
        w = round(int(wafer)/Qty.groupby(['WS Group']).size()[i],2)
        wafer_m.append(w)
    else:
        wafer_m.append(0)
    #print(i)
#print ('Number of wafers in each machine:',wafer_m)

Please enter the quantity of wafers:7000


In [166]:
col = ['3','4','7','8','9','C','D','E']
#print(wafer_in_each_machine)
wafer_in_each_machine = pd.DataFrame(columns = col)
machine_num = pd.DataFrame(columns = col)
for i in loc_change_group:
    for j in range(3):
        f1 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '3')].shape[0]
        f2 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '4')].shape[0]
        f3 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '7')].shape[0]
        f4 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '8')].shape[0]
        f5 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == '9')].shape[0]
        f6 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == 'C')].shape[0]
        f7 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == 'D')].shape[0]
        f8 = Qty[(Qty['WS Group'] == i[j]) & (Qty['Location'] == 'E')].shape[0]
        machine = np.array([f1,f2,f3,f4,f5,f6,f7,f8])
        machine_D = pd.DataFrame([machine],columns = col)
        machine_num = machine_num.append(machine_D)
        
        for k in range(8):
            if i[j] in WS_Group:
                num = wafer_m[step.index(i[j])]*machine[k]
            else:
                num = 0
            machine[k] = num
        machine = pd.DataFrame([machine],columns = col)
        #print(machine)
        wafer_in_each_machine = wafer_in_each_machine.append(machine)

#print(wafer_in_each_machine.shape[0])        

In [167]:
data = np.vsplit(wafer_in_each_machine.values, len(loc_change_group))

In [168]:
import itertools
col1 = ['3','4','7','8','9','C','D','E']
print(list(itertools.product(col, col1)))

[('3', '3'), ('3', '4'), ('3', '7'), ('3', '8'), ('3', '9'), ('3', 'C'), ('3', 'D'), ('3', 'E'), ('4', '3'), ('4', '4'), ('4', '7'), ('4', '8'), ('4', '9'), ('4', 'C'), ('4', 'D'), ('4', 'E'), ('7', '3'), ('7', '4'), ('7', '7'), ('7', '8'), ('7', '9'), ('7', 'C'), ('7', 'D'), ('7', 'E'), ('8', '3'), ('8', '4'), ('8', '7'), ('8', '8'), ('8', '9'), ('8', 'C'), ('8', 'D'), ('8', 'E'), ('9', '3'), ('9', '4'), ('9', '7'), ('9', '8'), ('9', '9'), ('9', 'C'), ('9', 'D'), ('9', 'E'), ('C', '3'), ('C', '4'), ('C', '7'), ('C', '8'), ('C', '9'), ('C', 'C'), ('C', 'D'), ('C', 'E'), ('D', '3'), ('D', '4'), ('D', '7'), ('D', '8'), ('D', '9'), ('D', 'C'), ('D', 'D'), ('D', 'E'), ('E', '3'), ('E', '4'), ('E', '7'), ('E', '8'), ('E', '9'), ('E', 'C'), ('E', 'D'), ('E', 'E')]


In [173]:
def weight(distance,carry):
    total_carry = []
    for i in range(2):
        carry_weight = np.multiply(carry[i],distance)
        total_carry_step = np.sum(carry_weight)-np.trace(carry_weight) 
        total_carry.append(total_carry_step)
    return sum(total_carry)
weight_distance = np.array([[0,1,1,2,3,1,2,3],[1,0,2,3,4,2,3,4],[1,2,0,1,2,1,2,3],[2,3,1,0,1,2,3,4]
                           ,[3,4,2,1,0,3,4,5],[1,2,1,2,3,0,1,2],[2,3,2,3,4,1,0,1],[3,4,3,4,5,2,1,0]])
Carriage = []
for l in range(len(loc_change_group)):
    a = range(len(col))
    c_m = []
    for k in range(2):
        for j in a:
            for i in a:
                c_n = max(data[l][k,j]-data[l][k+1,j],0)*data[l][k+1,a]//(int(wafer)-data[l][k+1,j]+1) #避免分母為零
            c_m.append(c_n)      
    c_step_m = np.array(c_m)
    print(c_step_m)
    #print(c_step_m.shape)
    c_carry = np.vsplit(c_step_m, 2)
    for i in range(2):
        for j in range(8):
            c_carry[i][j][j] = min(data[l][i,j],data[l][i+1,j])
            c_carry = np.array(c_carry)
    Carriage.append(weight(distance = weight_distance,carry = c_carry))
Carriage = np.array([Carriage]).reshape(len(loc_change_group),1)
Carriage = pd.DataFrame(Carriage,columns = ['Relocation Carriage'])
Carriage.head(len(loc_change_group)+1)

[[0 237 0 475 475 0 0 475]
 [0 0 0 0 0 0 0 0]
 [0 380 0 761 761 0 0 761]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 142 0 285 285 0 0 285]
 [0 95 0 190 190 0 0 190]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [665 0 110 0 0 221 0 0]
 [0 0 0 0 0 0 0 0]
 [1332 0 221 0 0 444 0 0]
 [1332 0 221 0 0 444 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1332 0 221 0 0 444 0 0]]
[[0 237 0 475 475 0 0 475]
 [0 0 0 0 0 0 0 0]
 [0 380 0 761 761 0 0 761]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 142 0 285 285 0 0 285]
 [0 95 0 190 190 0 0 190]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 499 0 0 499 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 999 0 0 999 0 0]
 [0 0 999 0 0 999 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 999 0 0 999 0 0]]
[[0 237 0 475 475 0 0 475]
 [0 0 0 0 0 0 0 0]
 [0 380 0 761 761 0 0 761]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 142 0 285 285 0 0 285]
 [0 95 0 190 190 0 0 190]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 499 0 0 499 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 999 0 0 999 0 0]
 [0 0 999 0 0 999 

,Relocation Carriage
0,29864
1,28430
2,28430
3,29508
4,29508
5,29508
6,29508
7,28430
8,29864
9,29023


In [174]:
excel = pd.read_excel('template_result for 120S PC 7K.xlsx',sheet_name = None)
xls1 = pd.ExcelFile('template_result for 120S PC 7K.xlsx')
xls1.sheet_names
compare = excel.get('Sheet1')
compare = compare['Step carriage'].tolist()
b = [i-1 for i in loc_change]
origin = loc_change+b
origin = sorted(origin)
origin_carriage = []
for i in range(0,len(origin)-1,2):
    origin_carriage.append(compare[origin[i]] + compare[origin[i+1]])
origin_carriage = pd.DataFrame(origin_carriage)
Carriage['Original Carriage'] = origin_carriage
Carriage.loc['Total Carriage'] = Carriage.apply(lambda x: x.sum())
Carriage.head(len(loc_change_group)+1)

,Relocation Carriage,Original Carriage
0,29864,18183
1,28430,17141
2,28430,13148
3,29508,21860
4,29508,18022
5,29508,17141
6,29508,17629
7,28430,17629
8,29864,17629
9,29023,17629
